**NOTE** `phimats` environment should be used as kernel

In [1]:
import numpy as np
from importlib import reload

import PreProcessing
from PreProcessing import PreProcessing as PP, ReadMesh

from BoundaryConditions import *

import PostProcessing
from PostProcessing import WriteXDMF

## Read mesh

In [2]:
# Path to .inp mesh file
modelName = "../cracktip"

mesh = ReadMesh(modelName)
print(mesh)

<meshio mesh object>
  Number of points: 31977
  Number of cells:
    quad: 9872
    quad: 5856
    quad: 7920
    quad: 7920
  Point sets: Crack
  Cell sets: Crack


### Simulation data

In [3]:
# Simulation name
Simul = "CrackTip_EquilBC"
# Simulation type
SimulType = "MechTrapping"
# Element name
elementName = "quad"  
# Element sets
nElementSets = len(mesh.cell_sets_dict.keys())
# Number of steps
nSteps = 500

### Material data

In [14]:
# Trapping --------------

Erho = 20000  # Hydroge dislocation enthalpy (positive) [J/mol]

Vm = 7.09e-6  # Molar volume of Fe [m³/mol]
Vrho = 2*Vm   # Molar volume around dislocation  [m³/mol]
Vh = 2e-6     # Partial molar volume of hydrogen in Fe [m³]
N = 6          
R = 8.31      # Universal gas constant [J/(molK)]
T = 300       # Temperature [K]

theta_b = 3.453e-3*Vm/N
print(theta_b)

cL = theta_b*N/Vm  # Concentration  [mol/m³]
print("cL", cL)

Krho   = np.exp(Erho/(R*T))
theta_rho = theta_b*Krho #/(1+theta_b*Krho)

Vrho = 2*Vm
c_rho = theta_rho/Vrho
print("c_rho", c_rho)
Z_rho = c_rho/cL
print("Z_rho", Z_rho)
zeta_rho = R*T*np.log(Z_rho)
# zeta_rho = 0
print("zeta_rho", zeta_rho)

4.080295e-09
cL 0.003453
c_rho 0.8772567112212009
Z_rho 254.0563890012166
zeta_rho 13805.127722078518


In [15]:
# Diffusivity --------------

dt = 1/500   # Time increment [s]
s = 1      # Trapping capacity 
m = 0      # Dislocation diffusivity ratio

D0x = 1e-6; DQx = 10000  # Diffusivity m^2/s

D0x = 8.45e-8; DQx = 5000  # Diffusivity m^2/s
DL = D0x*np.exp(-DQx/(R*T))

print(DL)

1.1371790945295182e-08


In [16]:
Materials = {
    "Material_1" : {
	"D0x" 	: D0x,
 	"D0y" 	: D0x,
	"DQx" 	: DQx,
	"DQy" 	: DQx,
	"m"   	: m,
	"s"   	: s,
	"Vh"  	: Vh,
 	"zeta_rho"   : zeta_rho,
    }
}

Materials

{'Material_1': {'D0x': 8.45e-08,
  'D0y': 8.45e-08,
  'DQx': 5000,
  'DQy': 5000,
  'm': 0,
  's': 1,
  'Vh': 2e-06,
  'zeta_rho': np.float64(13805.127722078518)}}

### Apply boundary conditions

In [17]:
conBCs = []
exitNods = []

meshBC = meshio.read("../cracktip_BC.inp")

for iNod in meshBC.point_sets["ConBC"]:
    
    conBCs.append([iNod, cL])
    
BCmesh = WriteConBCs(Simul, elementName, mesh, conBCs)

### Initialize simulation object

In [18]:
reload(PreProcessing)
from PreProcessing import PreProcessing as PP

In [ ]:
SimulationData = {
    
    "Simul"            : Simul,
	"SimulType"		   : SimulType,
    "mesh"             : mesh,
    "elementName"      : elementName,
    "nElementSets"     : nElementSets,
    "presBCs"          : conBCs,
    "conB"             : cL,
	"exitNods"         : exitNods,
    "nSteps"           : nSteps,
    "Materials"        : Materials,
	"T"                : T,
	"dt"               : dt,
}

CrackTip_DiffBC = PP(SimulationData)
CrackTip_DiffBC.WriteInputFile()

HDF5 file 'CrackTip_EquilBC_in.hdf5' opened successfully in write mode.
HDF5 file closed successfully.


In [ ]:
CrackTip_DiffBC.WriteOutputFile(OVERWRITE=True)

In [21]:
WriteXDMF(Simul, elementName, nSteps+501, "Transport2D")